# Purposes to have Python Interface to C\C++
* Create Python API for existing C\C++ projects
* Speed up things (especially useful on platforms where optimized python packages
might not be available)
* Access to low-level HW and System capabilities from high-level language
* Ease UI (Command Line or Graphical) creation
* Test C\C++ code from python

# CFFI and Ctypes comparison
## Ctypes:
**Pros**
* Built-in module
* Some optimized modules relies on cyypes to access low-level objects representation or row menory

**Cons**
* Works only in binary (ABI) mode through libffi
* A lot of extra code required to describe functions, structures, types, etc.

## CFFI
**Pros**
* Both ABI and API level available
* Standalone ready for distribution Python library can be created
* No need to manually describe function signatures and types
* Better performance comparing to ctypes

**Cons**
* Requires compilation
* **Limitations in C code support**
    * Preprocessor directives are not supported\supported partially
    * *"For now some C99 constructs are not supported, but all C89 should be, including macros"*

# Install binary dependencies and required python modules:

* CMake
* CFFI
* Cython
* PyBind11
* numpy
* tabulate

In [1]:
from sys import platform
import logging as log
import subprocess

log.basicConfig(format='[ %(levelname)s ] %(message)s', level=log.DEBUG)

log.info("Checking for installed CMake...")
p = subprocess.run(['which', 'cmake'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
if p.returncode == 0:
    log.info(f"CMake executable found at {p.stdout.decode()}")
else:
    log.warn("CMake executable not found! Installing ... ")
    cmd = ['brew', 'install', 'cmake'] if platform == 'darwin' else ['apt', 'install', 'cmake']
    log.info(f"Running: {' '.join(cmd)}...")
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if p.returncode != 0:
        log.error(f"Failed to install cmake.\n"
                  f"Process STDOUT:\n{p.stdout.decode()}\n"
                  f"Process STDERR:\n{p.stderr.decode()}")
    else:
        log.info("Installation succeed!")

log.info("Installing required python modules...")
p = subprocess.run(['pip', 'install', '-r', 'requirements.txt'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
if p.returncode != 0:
    log.error(f"Failed to install required python modules.\n"
              f"Process STDOUT:\n{p.stdout.decode()}\n"
              f"Process STDERR:\n{p.stderr.decode()}")
else:
    log.info("Installation succeed")

[ INFO ] Checking for installed CMake...
[ INFO ] CMake executable found at /usr/local/bin/cmake

[ INFO ] Installing required python modules...
[ INFO ] Installation succeed


# C Library listing

Bellow you can find C library functions declaration. For more convenience listings with each function implmentation
will be added bellow right before using it from python.

c_library/library.h
```c
/*
CDefError: cannot parse "#include <stdio.h>"
<cdef source string>:1:1: Directives not supported yet

CFFI doesn't support directives in headers
For compatibility between ctypes and CFFI all includes moved to .c files
#include <stdio.h>
#include <string.h>
#include <unistd.h>
*/

int func_ret_int(int val);
double func_ret_double(double val);
char *func_ret_str(char *val);
char func_many_args(int val1, double val2, char val3, short val4);

void arr_minus_one(int *data, unsigned int arr_size);
int *gen_arr(unsigned int size);
void fill_arr(int *arr, unsigned int size);
void qsort_wrap(int *arr, unsigned int size, unsigned int el_size);
int comp (const int *a, const int *b);

char *do_nothing_function(char *some_str);
```

Here is brief description of each method and its purpose:
* Functions demonstrating working with basic types. Do nothing besides of receiving arguments of certain types,
printing basic information, which will show us that real C function was invoked, and returning some value.
```c
int func_ret_int(int val);
double func_ret_double(double val);
char *func_ret_str(char *val);
char func_many_args(int val1, double val2, char val3, short val4);
```
* Functions demonstrating how to manipulate with arrays:
    *  `void arr_minus_one(int *data, unsigned int arr_size);` - demonstrates how to update existing array
    * `int *gen_arr(unsigned int size);` - demonstrates dynamic memory allocation for creating new array
    * `void gen_arr(int *arr, unsigned int size);` - fill pre-allocated integer array `int *arr` of given size
    `unsigned int size` with integer values in range `[0, size - 1]`
    * `void qsort_wrap(int *arr, unsigned int size, unsigned int el_size);` - wraps standard quick sort algorithm from libc.
    Will be used for performance comparison between C, Python and mixed quick sort implementation
    * `int comp (const int *a, const int *b)` - C implementation of comparison function required by standard qsort
    function from libc
    * `char *do_nothing_function(char *some_str);` - dummy function receiving and returning same value to asses overhead
    of calling C functions from Python


# Building C library from sources.

In [2]:
import os
import shutil

TUTOR_ROOT = os.getcwd()
CMAKE_ROOT_FOLDER = os.path.join(TUTOR_ROOT, 'c_library',)
BUILD_FOLDER = os.path.join(CMAKE_ROOT_FOLDER, 'build')
if os.path.exists(BUILD_FOLDER):
    shutil.rmtree(BUILD_FOLDER)
os.mkdir(BUILD_FOLDER)

os.chdir(BUILD_FOLDER)
log.info("Running CMake...")
p = subprocess.run(['cmake', os.path.pardir], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
if p.returncode != 0:
    log.error("CMake run failed!\n"
              f"STDOUT:\n {p.stdout.decode()}\n"
              f"STDERR:\n {p.stderr.decode()}\n")
else:
    log.info("CMake files generation succeed!\n"
             f"STDOUT:\n {p.stdout.decode()}\n")

log.info("Running library building with make...")
p = subprocess.run(['make'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
if p.returncode != 0:
    log.error("Building failed!\n"
              f"STDOUT:\n {p.stdout.decode()}\n"
              f"STDERR:\n {p.stderr.decode()}\n")
else:
    log.info("Building suceed succeed!\n"
             f"STDOUT:\n {p.stdout.decode()}\n")

os.chdir(TUTOR_ROOT)

if platform in ("linux", "linux2"):
    C_LIB_NAME = "libbindings_demo.so"
elif platform == "darwin":
    C_LIB_NAME = "libbindings_demo.dylib"
else:
    raise OSError(f"Unsupported platform {platform}")
C_LIB_PATH = os.path.join(BUILD_FOLDER, C_LIB_NAME)
if not os.path.exists(C_LIB_PATH):
    raise FileNotFoundError(f"Compiled library not found at {C_LIB_PATH}")
else:
    log.info(f"Compiled library located at {C_LIB_PATH}")

[ INFO ] Running CMake...
[ INFO ] CMake files generation succeed!
STDOUT:
 -- The C compiler identification is AppleClang 12.0.0.12000031
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /Library/Developer/CommandLineTools/usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Configuring done
-- Generating done
-- Build files have been written to: /Users/mikhailtreskin/repos/py_bindings_tutorial/c_library/build


[ INFO ] Running library building with make...
[ INFO ] Building suceed succeed!
STDOUT:
 Scanning dependencies of target bindings_demo
[ 25%] Building C object CMakeFiles/bindings_demo.dir/basic_funcs.c.o
[ 50%] Building C object CMakeFiles/bindings_demo.dir/arrays.c.o
[ 75%] Building C object CMakeFiles/bindings_demo.dir/vars.c.o
[100%] Linking C shared library libbindings_demo.dylib
[100%] Built target bindings_demo


[ INFO ] Compiled library located at /Users/mikhailtreskin/

# Ctypes demo

[ctypes](https://docs.python.org/3/library/ctypes.html) is a foreign function library for Python. It provides C
compatible data types, and allows calling functions in DLLs or shared libraries. It can be used to wrap these libraries in pure Python.

What's stands for foreign function library? It's a library which implements foreign function interface (FFI) convention
between two certain languages - between C as a host language and Python as a guest language in case of ctypes.
Briefly, FFI is an interface that allows code written in one language to call code written in another language.
Ctypes rely on [libffi](https://github.com/libffi/libffi) library implementing FFI calling convention for C as a host language.

Ctypes allows to make calls to C functions and refer to C objects from Python in a relatively easy way. Let's go
through several examples to on how to interact with precompiled C library from Python using ctypes

First of all we have to load dynamic library in a following way (on Windows WINDLL call should be used instead CDLL)

In [3]:
import ctypes

lib = ctypes.CDLL(C_LIB_PATH)

And now we have an access to all of the functions mentioned above and declared in `c_library/library.h`.
Let's start from calling a simple functions consuming one or several "simple" arguments.

## Working with generic data types
*Functions listed bellow are defined in `c_library/basic_funcs.c`*

### Working with numeric types
Working with integers on example of pretty simple C function consuming integer argument performing logging print from C
code and then returns same integer value, which was provided as an input argument:
```c
int func_ret_int(int val) {
    printf("From C Library: func_ret_int: %d\n", val);
    return val;
}

double func_ret_double(double val) {
    printf("From C Library: func_ret_double: %f\n", val);
    return val;
}

```

To call functions from C library we have to describe it's signature in Python by specifying input arguments types
and return value type of desired C function. And then we can call it as an usual Python method.

In [4]:
# Jupyter notebook suppress default output from our C library
# so custom manager to capture stdout from C was added
from stdout_capture import capture_c_stdout

# Specifying return type of the func_ret_int C function
lib.func_ret_int.restype = ctypes.c_int
# Specifying input arguments type of the func_ret_int C function
lib.func_ret_int.argtypes = [ctypes.c_int, ]

# Same for double data type
lib.func_ret_double.restype = ctypes.c_double
lib.func_ret_double.argtypes = [ctypes.c_double]
with capture_c_stdout():
    print('From Python Script: func_ret_int: ', lib.func_ret_int(101))
    print('From Python Script: func_ret_double: ', lib.func_ret_double(12.123456789))

From C Library: func_ret_int: 101
From Python Script: func_ret_int:  101
From C Library: func_ret_double: 12.123457
From Python Script: func_ret_double:  12.123456789


### Working with strings (char arrays in C)
The same for char functions like:
```c
char * func_ret_str(char *val) {
    printf("From C Library: func_ret_str: %s\n", val);
    return val;
}
```

On Python level the only important aspects is that we have to encode input strings to bytearray and then also decode
return value

In [5]:
lib.func_ret_str.restype = ctypes.c_char_p
lib.func_ret_str.argtypes = [ctypes.c_char_p, ]
with capture_c_stdout():
    print('From Python Script: func_ret_str: ', lib.func_ret_str('Hello!'.encode('utf-8')).decode("utf-8"))

From C Library: func_ret_str: Hello!
From Python Script: func_ret_str:  Hello!


### Function of several arguments
Also nothing outstanding for functions with several input arguments with different data types, e.g.:

```c
char func_many_args(int val1, double val2, char val3, short val4) {
    printf("From C Library: func_many_args: int - %d, double - %f, char - %c, short - %d\n", val1, val2, val3, val4);
    return val3;
}
```
To call the function all input argument types have to be defined in proper order:

In [6]:
lib.func_many_args.restype = ctypes.c_char
lib.func_many_args.argtypes = [ctypes.c_int, ctypes.c_double, ctypes.c_char, ctypes.c_short]
with capture_c_stdout():
    print('From Python Script: func_many_args: ',
      lib.func_many_args(15, 18.1617, 'X'.encode('utf-8'), 32000).decode("utf-8"))

From C Library: func_many_args: int - 15, double - 18.161700, char - X, short - 32000
From Python Script: func_many_args:  X


### Accessing variables
Ctypes also allows to access and modify variables from C library. In `c_library/vars.h` we have defined following variables
```c
int a = 5;
double b = 5.12345;
char c = 'X';
```

To access variables from loaded C library we need ro use `in_dll()` class method of corresponding data type of variable
we going to access. `in_dll()` requires two input arguments - the 1st is an loaded library handler, and the 2nd is string
with variable name. E.g.:

In [7]:
a = ctypes.c_int.in_dll(lib, "a")
print('From Python Script: a: ', a.value)
b = ctypes.c_double.in_dll(lib, "b")
print('From Python Script: b: ', b.value)
c = ctypes.c_char.in_dll(lib, "c")
print('From Python Script: c: ', c.value.decode("utf-8"))


From Python Script: a:  5
From Python Script: b:  5.12345
From Python Script: c:  X


Variables values modification also allowed using usual assignment operator

In [8]:
print(f"'a' initial value: {a.value}")
a.value = 22
a = ctypes.c_int.in_dll(lib, "a")
print(f"'a' modified value: {a.value}")

'a' initial value: 5
'a' modified value: 22


## Working with arrays and interaction with numpy

OK, above were a simple examples which barely will be used in real life and were provided as introduction and way
to understand how ctypes works. But what about real life examples?

Bellow we will have a look on how to have a deal with C arrays basing on manipulation with numpy arrays.

### Updating existing array

*c_library/arrays.c*
```c
void arr_minus_one(int *data, unsigned int arr_size) {
    printf("From C Library:\n");
    for (int i = 0; i < arr_size; ++i) {
        data[i] = data[i] - 1;
        printf("%d ", data[i]);
    }
}
```

#### Getting pointer from numpy array

In [9]:
import numpy as np

arr = np.asarray([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=np.int32)
data_p = arr.ctypes.data_as(ctypes.POINTER(ctypes.c_int))
arr_size = arr.size
el_size = arr.itemsize
dtype = str(arr.dtype).encode('utf-8')

lib.arr_minus_one.restype = None
lib.arr_minus_one.argtypes = [ctypes.POINTER(ctypes.c_int), ctypes.c_uint]
print(f"Python numpy array:\n {arr}")
with capture_c_stdout():
    lib.arr_minus_one(data_p, arr_size, el_size, dtype)
print(f"Updated python numpy array:\n {arr}")

Python numpy array:
 [[1 2 3]
 [4 5 6]
 [7 8 9]]
Updated python numpy array:
 [[0 1 2]
 [3 4 5]
 [6 7 8]]


#### Getting pointer from python list

In [10]:
arr = [1, 2, 3, 4, 5, 6, 7, 8, 9]
arr_size = len(arr)
data_p = (ctypes.c_int * arr_size)(*arr)
print(f"Python list:\n {arr}")
with capture_c_stdout():
    lib.arr_minus_one(data_p, arr_size)
print(f"Updated python list:\n {arr}")

Python list:
 [1, 2, 3, 4, 5, 6, 7, 8, 9]
Updated python list:
 [1, 2, 3, 4, 5, 6, 7, 8, 9]


### Allocating dynamic array to create numpy array from the pointer

```c
int *gen_arr(unsigned int size) {
    int *arr = malloc(sizeof(int) * size);
    for (unsigned int i = 0; i < size; ++i) {
        arr[i] = i;
    }
    return arr;
}
```

In [11]:
lib.gen_arr.restype = ctypes.POINTER(ctypes.c_int)
lib.gen_arr.argtypes = [ctypes.c_int, ]
data_p = lib.gen_arr(10)

# Create from ctypes array
ctypes_arr = (ctypes.c_int * 10).from_address(ctypes.addressof(data_p.contents))
flat_arr = np.ctypeslib.as_array(ctypes_arr, shape=(2, 5))  # Shape ignored
print(f"Flat numpy array:\n {flat_arr}")
# Create from pointer directly
shaped_array = np.ctypeslib.as_array(data_p, shape=(5, 2))
print(f"Shaped numpy array:\n {shaped_array}")


Flat numpy array:
 [0 1 2 3 4 5 6 7 8 9]
Shaped numpy array:
 [[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]


### Fill pre-allocated memory

#### Write to ctypes array

```c
void fill_arr(int *arr, unsigned int size) {
    for (int i = 0; i < size; ++i) {
        arr[i] = i;
    }
}
```

In [12]:
lib.gen_arr.argtypes = [ctypes.POINTER(ctypes.c_int), ctypes.c_uint]
lib.gen_arr.restype = None

# Allocating memory
i_arr = (ctypes.c_int * 15)()
i_p = ctypes.cast(i_arr, ctypes.POINTER(ctypes.c_int))
lib.fill_arr(i_p, 15)
# Build numpy array
shaped_array = np.ctypeslib.as_array(i_p, shape=(3, 5))
print(f"Resulting numpy array:\n {shaped_array}")



Resulting numpy array:
 [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]]


#### Write directly to numpy array's memory

In [13]:
arr = np.ndarray(shape=(5, 5), dtype=np.int32)
arr_p = arr.ctypes.data_as(ctypes.POINTER(ctypes.c_int))
lib.fill_arr(arr_p, 25)
print(f"Shaped numpy array:\n {arr}")

Shaped numpy array:
 [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]


## Find library and callbacks

Ctypes provides functionality allowing to locate a library in a way similar to what the compiler or runtime loader
does (on platforms with several versions of a shared library the most recent should be loaded)
To search fo a library in a default paths `find_library()` method should be called with library name without extension
name or 'lib' prefix on Linux\MacOS. (In example bellow 'c' library name stands for `libc.so` or `libc.dyli`
depending on platform)

We will use `qsort` function from standard C library performing quick sort of an array.
The function requires following arguments: array to sort, array length, element size, and comparing callback function.


In case bellow we will pass Python function as sorting callback to `qsort` function to demonstrate Python function
calls from C code mechanism using ctypes library.
The callback will then be called with two pointers to items, and it must return a negative integer if the first
item is smaller than the second, a zero if they are equal, and a positive integer otherwise.

In [14]:
from ctypes.util import find_library
# libc and find_library
libc = ctypes.CDLL(find_library('c'))

a = (ctypes.c_int * 10)(5, 1, 7, 33, 99, 43, 12, 5, 1, 0)
libc.qsort.restype = None

# Creating callback function using CFUNCTYPE function factory decorator
@ctypes.CFUNCTYPE(ctypes.c_int, ctypes.POINTER(ctypes.c_int), ctypes.POINTER(ctypes.c_int))
def py_cmp_func(a, b):
    return a[0] - b[0]

# Calling C qsort passing python function as sorting callback
libc.qsort(a, len(a), ctypes.sizeof(ctypes.c_int), py_cmp_func)
print([v for v in a])

[0, 1, 1, 5, 5, 7, 12, 33, 43, 99]


## Some performance comparison

Bellow we will compare calling of quick sort in a three different modes:
* Pure python quick sort implementation
* Calling C qsort implementation with Python comparison callback
* Pure C qsort with C comparison callback implementation

In [15]:
from timeit import default_timer
import random

def py_qsort(nums):
    if len(nums) <= 1:
        return nums
    else:
        q = random.choice(nums)
    l_nums = [n for n in nums if n < q]

    e_nums = [q] * nums.count(q)
    b_nums = [n for n in nums if n > q]
    return py_qsort(l_nums) + e_nums + py_qsort(b_nums)

rand_arr = np.random.randint(low=0, high=500000, size=1000000, dtype=np.int32)

rand_list = list(rand_arr)
t0 = default_timer()
sorted_arr = py_qsort(rand_list)
t1 = default_timer()
print(f"Python qsort time: {t1 - t0}")

rand_data_p = rand_arr.ctypes.data_as(ctypes.POINTER(ctypes.c_int))
t0 = default_timer()
libc.qsort(rand_data_p, rand_arr.size, ctypes.sizeof(ctypes.c_int), py_cmp_func)
t1 = default_timer()
print(f"libc qsort with Python callback time: {t1 - t0}")

rand_arr = np.random.randint(low=0, high=500000, size=1000000, dtype=np.int32)
rand_data_p = rand_arr.ctypes.data_as(ctypes.POINTER(ctypes.c_int))
lib.qsort_wrap.restype = None
t0 = default_timer()
lib.qsort_wrap(rand_data_p, rand_arr.size, ctypes.sizeof(ctypes.c_int))
t1 = default_timer()
print(f"Pure C qsort: {t1 - t0}")

Python qsort time: 4.8456917
libc qsort with Python callback time: 10.829024537000002
Pure C qsort: 0.13771502599999863


# TRACE FROM VTUNE TO BE ADDED


# CFFI demo

CFFI library provides similar with ctypes functionality and also extends it with several modes of library usage.

**Main advantages of cffi library:**
* The goal is to call C code from Python without learning a 3rd language: existing alternatives require users to
learn domain specific language (Cython, SWIG) or API (ctypes). The CFFI design requires users to know only C and Python,
minimizing the extra bits of API that need to be learned.
* Keep all the Python-related logic in Python so that you don’t need to write much C code
(unlike CPython native C extensions).
* Functions signatures and types definition performed automatically basing on headers parsing.
User don't have to describe function arguments and return value, structure fields etc.
* API and ABI usage modes

**Main disadvantages:**
* Preprocessors directives are not supported which force to implement single-header library since `#include`
directives are not allowed on header parsing stage.
* Little bit complicated building process especially in API mode.

## In-Line ABI Mode

The simplest mode from implementation perspective.
This mode is pretty the same as `ctypes.CDLL()` which opens precompiled library and works on ABI (
Application Binary Interface) level.

In [16]:
# Parsing header file to get types and signatures
import cffi
ffi = cffi.FFI()
include = os.path.join("c_library", "library.h")
with open(include) as f:
    ffi.cdef(f.read())

Opening precompiled library and accessing functions. No need to describe function signature.
We will measure library loading time to compare In-Line and Out-of-Line ABI modes bellow

In [17]:
t0 = default_timer()
lib = ffi.dlopen(C_LIB_PATH)
t1 = default_timer()
log.info(f"In-line ABI loading time {t1 - t0}")
with capture_c_stdout():
    res = lib.func_ret_str("Some str".encode("utf-8"))
    print(f"From Python Script: {ffi.string(res).decode('utf-8')}")

[ INFO ] In-line ABI loading time 0.00015859500000203752


From C Library: func_ret_str: Some str
From Python Script: Some str


## Out-of-Line ABI Mode

As in In-Line ABI mode, opens precompiled library with further compilation to create python module
with binary info which speed-ups import time.

No real compilation happens (C compiler doesn't invoked) since we doesn't provide any sources in `ffi.set_source()` call

In [18]:
# Need to reload cffi module and ffi object to allow to redefine header file and sources
# (Such trick required only for this jupyter notebook)
if "cffi" in globals():
    globals().pop("cffi")
if "ffi" in globals():
    globals().pop("ffi")
import cffi
ffi = cffi.FFI()
with open(include) as f:
    ffi.cdef(f.read())

# Set destination package structure
ffi.set_source("cffi_libs.out_of_line_abi._bindings_demo", None)
# Compile
ffi.compile()
target_module = "./cffi_libs/out_of_line_abi/_bindings_demo.py"
if not os.path.exists(target_module):
    raise FileNotFoundError(f"{target_module} not found! Possibly CFFI module compilation failed!")
log.info("Content of resulting folder")
print("./cffi_libs/out_of_line_abi/:")
for entry in os.listdir("./cffi_libs/out_of_line_abi/"):
    print(f"    {entry}")

[ INFO ] Content of resulting folder


./cffi_libs/out_of_line_abi/:
    __pycache__
    _bindings_demo.py


Unlike In-Line ABI mode we will create `FFI` object not from `cffi` module but import it from our compiled above package
(stored in `ffi` variable).


In [19]:
from cffi_libs.out_of_line_abi._bindings_demo import ffi as out_of_line_abi_ffi
t0 = default_timer()
lib = out_of_line_abi_ffi.dlopen(C_LIB_PATH)
t1 = default_timer()
log.info(f"Out-of-line ABI import time {t1 - t0}")
with capture_c_stdout():
    res = lib.func_ret_str("Some str".encode("utf-8"))
    print(f"From Python Script: {ffi.string(res).decode('utf-8')}")

[ INFO ] Out-of-line ABI import time 0.0001150940000016476


From C Library: func_ret_str: Some str
From Python Script: Some str


As we can see Out-of-line ABI mode speed-up library opening comparing with In-Line ABI and also reduce ffi calls
overhead (will see bellow)

## Out-of-Line API Mode
**Preferable mode**

This mode requires passing library sources to `cffi.set_source()` function to perform importable python library
compilation. On compilation stage CFFI generates source code which wraps methods from provided in
C library (or from source code) with required CPython code and then invokes C compiler to produce importable
python library. This mode results to the best performance but requires preliminary compilation.

In [20]:
if "cffi" in globals():
    globals().pop("cffi")
if "ffi" in globals():
    globals().pop("ffi")
import cffi
ffi = cffi.FFI()
with open(include) as f:
    ffi.cdef(f.read())
module_name = "_bindings_demo"
ffi.set_source(module_name,  # name of compiled library
               # Source code to be compiled along with CFFI-generated wrapper code
               f'#include "../../{include}"',
               # Libraries to link compiled CFFI module with (no extension and 'lib' prefix)
               # Passed directly to compiler as '-l' option
               libraries=["bindings_demo"],
               library_dirs=[os.path.dirname(C_LIB_PATH)],
               )
target_dir = './cffi_libs/out_of_line_api'
ffi.compile(tmpdir=target_dir, verbose=True)

log.info("Content of resulting folder")
print(target_dir)
target_module_found = False
library_name = None
for entry in os.listdir(target_dir):
    print(f"    {entry}")
    if entry.startswith(module_name) and entry.endswith(".so"):  # both Linux and MacOS creates .so file
        library_name = entry
        target_module_found = True

if not target_module_found:
    raise FileNotFoundError(f"{target_module} library not found! Possibly CFFI module compilation failed!")

generating ./cffi_libs/out_of_line_api/_bindings_demo.c
(already up-to-date)
setting the current directory to '/Users/mikhailtreskin/repos/py_bindings_tutorial/cffi_libs/out_of_line_api'
running build_ext
building '_bindings_demo' extension
clang -Wno-unused-result -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -isysroot /Library/Developer/CommandLineTools/SDKs/MacOSX10.15.sdk -I/Library/Developer/CommandLineTools/SDKs/MacOSX10.15.sdk/usr/include -I/Library/Developer/CommandLineTools/SDKs/MacOSX10.15.sdk/System/Library/Frameworks/Tk.framework/Versions/8.5/Headers -I/usr/local/include -I/usr/local/opt/openssl@1.1/include -I/usr/local/opt/sqlite/include -I/Users/mikhailtreskin/repos/py_bindings_tutorial/venv/include -I/usr/local/Cellar/python@3.8/3.8.5/Frameworks/Python.framework/Versions/3.8/include/python3.8 -c _bindings_demo.c -o ./_bindings_demo.o
clang -bundle -undefined dynamic_lookup -isysroot /Library/Developer/CommandLineTools/SDKs/MacOSX10.

[ INFO ] Content of resulting folder


Note that compiled python module dynamically linked against initial 'bindings_demo' library

In [21]:
if platform == "darwin":
    command = ['otool', '-L', os.path.join(target_dir, library_name)]
elif platform.startswith("linux"):
    command = ['ldd', os.path.join(target_dir, library_name)]
p = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

if p.returncode != 0:
    log.error("otool execution failed!\n"
              f"STDOUT:\n {p.stdout.decode()}\n"
              f"STDERR:\n {p.stderr.decode()}\n")
else:
    log.info(f"{library_name} library dependencies:\n"
             f"{p.stdout.decode()}\n")

[ INFO ] _bindings_demo.cpython-38-darwin.so library dependencies:
./cffi_libs/out_of_line_api/_bindings_demo.cpython-38-darwin.so:
	/Users/mikhailtreskin/repos/py_bindings_tutorial/c_library/build/libbindings_demo.dylib (compatibility version 0.0.0, current version 0.0.0)
	/usr/lib/libSystem.B.dylib (compatibility version 1.0.0, current version 1281.100.1)




Compiled module usage example

In [22]:
from cffi_libs.out_of_line_api._bindings_demo import ffi as out_of_line_api_ffi
from cffi_libs.out_of_line_api._bindings_demo import lib as out_of_line_api_lib

# dlopen not required
with capture_c_stdout():
    res = out_of_line_api_lib.func_ret_str("Some str".encode("utf-8"))
    print(f"From Python Script: {out_of_line_api_ffi.string(res).decode('utf-8')}")

From C Library: func_ret_str: Some str
From Python Script: Some str


## Some extra usage examples

### Declaring C source right in set_source() method
Another possible way of using Out-of-Line API mode which allows to avoid dynamic dependencies
is to specify all C sources right in `ffi.set_source()` method and do not add dependency libraries.
``` python
ffi.set_source("_my_module",  # name of compiled library
               # Source code to be compiled along with CFFI-generated wrapper code
               '''
                #include "library.h"

                #include <stdio.h>
                #include <string.h>
                #include <unistd.h>

                int func_ret_int(int val) {
                    printf("From C Library: func_ret_int: %d\n", val);
                    return val;
                }
                ''',
               # Libraries to link compiled CFFI module with (no extension and 'lib' prefix)
               # Passed directly to compiler as '-l' option
               libraries=[],
               )
```

### Setuptools and distutils integration

Finally, you can (but don’t have to) use CFFI’s Distutils or Setuptools integration when writing a setup.py.

For Distutils (only in out-of-line API mode):
``` python
# setup.py (requires CFFI to be installed first)
from distutils.core import setup

import foo_build   # possibly with sys.path tricks to find it

setup(
    ...,
    ext_modules=[foo_build.ffibuilder.distutils_extension()],
)
```

For Setuptools (out-of-line, but works in ABI or API mode; recommended):
``` python
# setup.py (with automatic dependency tracking)
from setuptools import setup

setup(
    ...,
    setup_requires=["cffi>=1.0.0"],
    cffi_modules=["package/foo_build.py:ffibuilder"],
    install_requires=["cffi>=1.0.0"],
)
```
Note again that the foo_build.py example contains the following lines, which mean that the ffi builder is not
actually compiled when package.foo_build is merely imported—it will be compiled independently by the Setuptools logic,
using compilation parameters provided by Setuptools:

``` python
if __name__ == "__main__":    # not when running with setuptools
    ffibuilder.compile(verbose=True)
```

### Partial C code declaration

Using `...` in `ffi.cdef()` allows to skip some part of declarations in case of explicit declarations specification
(as a multiline string) and 'ask' C compiler to fill the gaps. Such way of declaration can be convenient in some
cases e.g. for `struct{}` or `union{}` declaration.
More details in the [documentation](https://cffi.readthedocs.io/en/latest/cdef.html#letting-the-c-compiler-fill-the-gaps).

# FFI overhead comparison

In order to compare overheads of calling C functions from python in ctypes and CFFI (different modes)
we will run `char *do_nothing_function(char *some_str)` C function which literally doesn't do anything
except receiving and returning some value.
In such approach all of the calling time will be spent on accessing C function from Python code which allows to asses
only FFI interface overhead.

In [23]:
from  tabulate import tabulate
ITERATIONS_COUNT = 500000
c_types_lib = ctypes.CDLL(C_LIB_PATH)

ffi = cffi.FFI()
with open(include) as f:
    ffi.cdef(f.read())
cffi_in_line_abi_lib = ffi.dlopen(C_LIB_PATH)

from cffi_libs.out_of_line_abi._bindings_demo import ffi as out_of_line_abi_ffi
cffi_out_of_line_abi_lib = out_of_line_abi_ffi.dlopen(C_LIB_PATH)

from cffi_libs.out_of_line_api._bindings_demo import lib as out_of_line_api_lib

c_types_times = []
for i in range(ITERATIONS_COUNT):
    t0 = default_timer()
    res = c_types_lib.do_nothing_function("Some str".encode("utf-8"))
    c_types_times.append(default_timer() - t0)

cffi_in_line_abi_lib_times = []
for i in range(ITERATIONS_COUNT):
    t0 = default_timer()
    res = cffi_in_line_abi_lib.do_nothing_function("Some str".encode("utf-8"))
    cffi_in_line_abi_lib_times.append(default_timer() - t0)

cffi_out_of_line_abi_times = []
for i in range(ITERATIONS_COUNT):
    t0 = default_timer()
    res = cffi_out_of_line_abi_lib.do_nothing_function("Some str".encode("utf-8"))
    cffi_out_of_line_abi_times.append(default_timer() - t0)

cffi_out_of_line_api_times = []
for i in range(ITERATIONS_COUNT):
    t0 = default_timer()
    res = out_of_line_api_lib.do_nothing_function("Some str".encode("utf-8"))
    cffi_out_of_line_api_times.append(default_timer() - t0)

ctypes_mean = np.mean(c_types_times)
cffi_in_line_abi_mean = np.mean(cffi_in_line_abi_lib_times)
cffi_out_of_line_abi_mean = np.mean(cffi_out_of_line_abi_times)
cffi_out_of_line_api_mean = np.mean(cffi_out_of_line_api_times)

fastest_mean = np.min([ctypes_mean, cffi_in_line_abi_mean, cffi_out_of_line_abi_mean, cffi_out_of_line_api_mean])
headers = ["Library", "Mean", "Ratio"]
results = [
    ["ctypes", ctypes_mean, fastest_mean / ctypes_mean],
    ["cffi in-line", cffi_in_line_abi_mean, fastest_mean / cffi_in_line_abi_mean],
    ["cffi out-of-line abi", cffi_out_of_line_abi_mean, fastest_mean / cffi_out_of_line_abi_mean],
    ["cffi out-of-line api", cffi_out_of_line_api_mean, fastest_mean / cffi_out_of_line_api_mean],
]
print(tabulate(results, headers=headers, tablefmt='fancy_grid'))

╒══════════════════════╤═════════════╤══════════╕
│ Library              │        Mean │    Ratio │
╞══════════════════════╪═════════════╪══════════╡
│ ctypes               │ 5.58395e-07 │ 0.732638 │
├──────────────────────┼─────────────┼──────────┤
│ cffi in-line         │ 4.74789e-07 │ 0.861649 │
├──────────────────────┼─────────────┼──────────┤
│ cffi out-of-line abi │ 4.5045e-07  │ 0.908207 │
├──────────────────────┼─────────────┼──────────┤
│ cffi out-of-line api │ 4.09101e-07 │ 1        │
╘══════════════════════╧═════════════╧══════════╛


# SWiG overview

*Pros*
* Multi-language bindings library (Python, Java, C#, Perl etc. support)
* Elegant Python interface

*Cons*
* Explicit compilation required or not really obvious CMake integration
([tutorial here](https://github.com/danielunderwood/swig-example))
* Need to create separate interface file with SWIG-own syntax
* Explicit template instantiations required for C++
* Only API mode

## Example (from docs)

*Header file  example*
```cpp
// pair.h.  A pair like the STL
 namespace std {
    template<class T1, class T2> struct pair {
        T1 first;
        T2 second;
        pair() : first(T1()), second(T2()) { };
        pair(const T1 &f, const T2 &s) : first(f), second(s) { }
    };
 }
```
*SWIG interface file*
```
 // pair.i - SWIG interface
 %module pair
 %{
 #include "pair.h"
 %}

 // Ignore the default constructor
 %ignore std::pair::pair();

 // Parse the original header file
 %include "pair.h"

 // Instantiate some templates

 %template(pairii) std::pair<int,int>;
 %template(pairdi) std::pair<double,int>;
```

*Compilation example*

```shell
swig -python -c++ pair.i
g++ -c -fpic pair_wrap.c -I/usr/include/python3.8
g++ -shared pair_wrap.o -o _pair.so
```

*Python usage example*

```python
import pair
a = pair.pairii(3,4)
print(a.first)
print(a.second)
a.second = 16
print(a.second)
b = pair.pairdi(3.5,8)
print(b.first)
print(b.second)
```

# Boost-Python
Outdated library, up-to-date alternative is pybind11.

**From pybind11 docs:**

*"The main issue with Boost.Python—and the reason for creating such a similar project—is Boost.
Boost is an enormously large and complex suite of utility libraries that works with almost every C++ compiler
in existence. This compatibility has its cost: arcane template tricks and workarounds are necessary to support the
oldest and buggiest of compiler specimens. Now that C++11-compatible compilers are widely available,
this heavy machinery has become an excessively large and unnecessary dependency."*

[Tutorial example](https://habr.com/ru/post/168083/)